In [1]:
import pandas as pd
import numpy as np
import tarfile
import os


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
DATA_DIR = 'data' #'drive/My Drive/datasets/' #@param {type:"string"}
FILE_NAME = 'gun_violence' #@param {type:"string"}

file = tarfile.open(os.path.join(DATA_DIR,f'{FILE_NAME}.tar.gz'))
file.extractall('./data')
file.close()


In [4]:
# https://github.com/jamesqo/gun-violence-data
DATOS = pd.read_csv('./data/stage3.csv')
DATOS.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


Veamos el número de registros vacíos por cada columna:

In [ ]:
# 240k filas
DATOS.isna().sum()


incident_id                         0
date                                0
state                               0
city_or_county                      0
address                         16497
n_killed                            0
n_injured                           0
incident_url                        0
source_url                        468
incident_url_fields_missing         0
congressional_district          11944
gun_stolen                      99498
gun_type                        99451
incident_characteristics          326
latitude                         7923
location_description           197588
longitude                        7923
n_guns_involved                 99451
notes                           81017
participant_age                 92298
participant_age_group           42119
participant_gender              36362
participant_name               122253
participant_relationship       223903
participant_status              27626
participant_type                24863
sources     

Eliminamos las columnas con mayor cantidad de valores nulos (participant_relationship, location_description, participant_name), que son prescindibles o contienen información personal que no nos interesa. 

Eliminamos también incident_url_fields_missing que siempre tiene valor False y las columnas que tienen las url de las noticias a excepción de incident_url que no tiene ningún valor faltante (sources, source_url). 

Por último, prescindimos también de las columnas state_house_district y state_senate_district, a favor de congressional_district, que es la que menor número de valores nulos tiene y porque las tres columnas se refieren a lo mismo: la entidad territorial que elige a un solo miembro del congreso.

In [10]:
Columns2Delete = ["participant_relationship","location_description","participant_name","incident_url_fields_missing","sources","source_url","state_house_district","state_senate_district"]
for c in Columns2Delete:
  del DATOS[c]
DATOS.isna().sum()

incident_id                     0
date                            0
state                           0
city_or_county                  0
address                     16497
n_killed                        0
n_injured                       0
incident_url                    0
congressional_district      11944
gun_stolen                  99498
gun_type                    99451
incident_characteristics      326
latitude                     7923
longitude                    7923
n_guns_involved             99451
notes                       81017
participant_age             92298
participant_age_group       42119
participant_gender          36362
participant_status          27626
participant_type            24863
dtype: int64

Una vez eliminadas las columnas con mayor número de valores faltantes y aquellas que no nos son de interés, procedemos a dividir el dataset en tres subdatasets, cada uno de los cuales será tratado en cuanto a valores nulos y faltantes; en lugar de hacer esto antes de dividirlo pues se perdería información.

In [11]:
DATOS = DATOS.dropna()
DATOS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55226 entries, 4 to 239676
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   incident_id               55226 non-null  int64  
 1   date                      55226 non-null  object 
 2   state                     55226 non-null  object 
 3   city_or_county            55226 non-null  object 
 4   address                   55226 non-null  object 
 5   n_killed                  55226 non-null  int64  
 6   n_injured                 55226 non-null  int64  
 7   incident_url              55226 non-null  object 
 8   congressional_district    55226 non-null  float64
 9   gun_stolen                55226 non-null  object 
 10  gun_type                  55226 non-null  object 
 11  incident_characteristics  55226 non-null  object 
 12  latitude                  55226 non-null  float64
 13  longitude                 55226 non-null  float64
 14  n_gun

In [12]:
DATOS.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,congressional_district,gun_stolen,...,incident_characteristics,latitude,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_status,participant_type
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,6.0,0::Unknown||1::Unknown,...,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
10,491674,2013-01-23,Tennessee,Chattanooga,1501 Dodds Ave,1,3,http://www.gunviolencearchive.org/incident/491674,3.0,0::Unknown,...,"Shot - Wounded/Injured||Shot - Dead (murder, a...",35.0221,-85.2697,1.0,19 yr. old male dies; 3 inured; shooting on Do...,0::19,0::Adult 18+,0::Male||1::Male||2::Male||3::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
11,479413,2013-01-25,Missouri,Saint Louis,W Florissant Ave and Riverview Blvd,1,3,http://www.gunviolencearchive.org/incident/479413,1.0,0::Unknown,...,"Shot - Wounded/Injured||Shot - Dead (murder, a...",38.7067,-90.2494,1.0,"38.706732, -90.249375",0::28,0::Adult 18+,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
12,479561,2013-01-26,Louisiana,Charenton,1000 block of Flat Town Road,2,3,http://www.gunviolencearchive.org/incident/479561,3.0,0::Unknown,...,"Shot - Wounded/Injured||Shot - Dead (murder, a...",29.8816,-91.5251,1.0,Ofc. hailed from Chitimacha Tribe of Louisiana...,3::78||4::48,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Killed||1::Injured||2::Injured||3::Killed||...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
16,479580,2013-02-03,California,Yuba (county),5800 block of Poplar Avenue,1,3,http://www.gunviolencearchive.org/incident/479580,3.0,0::Unknown,...,"Shot - Wounded/Injured||Shot - Dead (murder, a...",39.1236,-121.5830,1.0,"perps have gang affiliation, but unclear as to...",0::20||4::25||5::18||6::19,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Female||4::Male||5::Male|...,0::Killed||1::Injured||2::Injured||3::Injured|...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
